<a href="https://colab.research.google.com/github/23AD083/MLOPS_INTERNSHIP/blob/main/CROP_PREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DAY 11 - 16/07/2025


STEP 01 ) **PROBLEM** DEFINITION - CROP CULTIVATION SUGGESTION

In [ ]:

# Libraries for data manipulation and analysis
import pandas as pd
import numpy as np

# Libraries for machine learning model building and evaluation
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


STEP 02 ) DATA CALLOECTION FROM THE GIVE CSV **FILE**

In [ ]:


from google.colab import drive
drive.mount('/content/drive')

# Replace 'your_file_name.csv' with the actual path to your CSV file in Google Drive
file_path = '/content/cropdata.csv'
df = pd.read_csv(file_path)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


STEP 03 ) PREPROCESSING **bold text**

In [ ]:
# prompt: give the code for the head , info , desc , null value , outlier

# head
print("Head of the dataframe:")
print(df.head())



Head of the dataframe:
    N   P   K  temperature   humidity        ph    rainfall label
0  90  42  43    20.879744  82.002744  6.502985  202.935536  rice
1  85  58  41    21.770462  80.319644  7.038096  226.655537  rice
2  60  55  44    23.004459  82.320763  7.840207  263.964248  rice
3  74  35  40    26.491096  80.158363  6.980401  242.864034  rice
4  78  42  42    20.130175  81.604873  7.628473  262.717340  rice


In [ ]:

# info
print("\nInformation about the dataframe:")
df.info()



Information about the dataframe:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   N            2200 non-null   int64  
 1   P            2200 non-null   int64  
 2   K            2200 non-null   int64  
 3   temperature  2200 non-null   float64
 4   humidity     2200 non-null   float64
 5   ph           2200 non-null   float64
 6   rainfall     2200 non-null   float64
 7   label        2200 non-null   object 
dtypes: float64(4), int64(3), object(1)
memory usage: 137.6+ KB


In [ ]:

# describe
print("\nStatistical description of the dataframe:")
print(df.describe())



Statistical description of the dataframe:
                 N            P            K  temperature     humidity  \
count  2200.000000  2200.000000  2200.000000  2200.000000  2200.000000   
mean     50.551818    53.362727    48.149091    25.616244    71.481779   
std      36.917334    32.985883    50.647931     5.063749    22.263812   
min       0.000000     5.000000     5.000000     8.825675    14.258040   
25%      21.000000    28.000000    20.000000    22.769375    60.261953   
50%      37.000000    51.000000    32.000000    25.598693    80.473146   
75%      84.250000    68.000000    49.000000    28.561654    89.948771   
max     140.000000   145.000000   205.000000    43.675493    99.981876   

                ph     rainfall  
count  2200.000000  2200.000000  
mean      6.469480   103.463655  
std       0.773938    54.958389  
min       3.504752    20.211267  
25%       5.971693    64.551686  
50%       6.425045    94.867624  
75%       6.923643   124.267508  
max       9.935091

In [ ]:

# null values
print("\nNumber of null values per column:")
print(df.isnull().sum())

# outlier detection (using IQR method as an example)
# For numerical columns, identify potential outliers
numerical_cols = df.select_dtypes(include=np.number).columns


Number of null values per column:
N              0
P              0
K              0
temperature    0
humidity       0
ph             0
rainfall       0
label          0
dtype: int64


In [ ]:
# Assuming 'label' is the column name indicating the crop type
# Assuming 'df' is your pandas DataFrame loaded from the CSV

# Separate data by crop type
df_by_crop = {crop: df[df['label'] == crop] for crop in df['label'].unique()}


# Determine the minimum number of samples per crop
min_samples = min(len(df_by_crop[crop]) for crop in df_by_crop)

# Sample the minimum number of samples from each crop and concatenate
balanced_df = pd.concat([df_by_crop[crop].sample(min_samples, random_state=42) for crop in df_by_crop])

# Shuffle the balanced dataframe
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Now, split the balanced_df into train and test sets
X = balanced_df.drop('label', axis=1) # Assuming 'label' is the target variable
y = balanced_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("\nBalanced and Shuffled Dataframe:")
print(balanced_df.head())


Balanced and Shuffled Dataframe:
     N    P    K  temperature   humidity        ph    rainfall       label
0  118   18   52    28.049436  90.831307  6.562833   20.762230   muskmelon
1   95   14   50    26.633312  84.317568  6.560444   56.318662  watermelon
2   39   64   53    23.012402  91.073555  6.598860  208.335798      papaya
3   70   68   45    33.835086  92.854702  6.991626  203.404403      papaya
4   29  138  197    22.190554  92.437642  5.830892  121.662276       apple


In [33]:
# prompt: print the labeled data

print("\nLabels in the dataset:")
print(y.unique())

print("\nTraining labels distribution:")
print(y_train.value_counts())

print("\nTesting labels distribution:")
print(y_test.value_counts())


Labels in the dataset:
['muskmelon' 'watermelon' 'papaya' 'apple' 'mango' 'mothbeans' 'mungbean'
 'lentil' 'blackgram' 'coconut' 'pomegranate' 'jute' 'maize' 'coffee'
 'orange' 'chickpea' 'pigeonpeas' 'rice' 'kidneybeans' 'grapes' 'cotton'
 'banana']

Training labels distribution:
label
orange         80
grapes         80
kidneybeans    80
mothbeans      80
cotton         80
banana         80
lentil         80
mungbean       80
coffee         80
muskmelon      80
apple          80
blackgram      80
pigeonpeas     80
maize          80
rice           80
watermelon     80
jute           80
mango          80
pomegranate    80
papaya         80
coconut        80
chickpea       80
Name: count, dtype: int64

Testing labels distribution:
label
orange         20
banana         20
cotton         20
maize          20
chickpea       20
rice           20
blackgram      20
watermelon     20
pomegranate    20
mothbeans      20
grapes         20
mango          20
apple          20
kidneybeans    20
j

In [30]:
# prompt: code for encoding the dataset with labeled data with labeled data

from sklearn.preprocessing import LabelEncoder

# Encode the target variable 'label'
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

print("\nEncoded Training Labels:")
print(y_train_encoded[:5])
print("\nEncoded Testing Labels:")
print(y_test_encoded[:5])


Encoded Training Labels:
[16  7  9 13 16]

Encoded Testing Labels:
[16  1  6 11 16]
